## Import packages

In [58]:
# Standard libraries
import requests
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2
from pandasql import sqldf

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# for env variables
import os
from dotenv import load_dotenv, get_key
load_dotenv()

# save env variables
SUPABASE_USER = get_key('.env', 'SUPABASE_USER')
SUPABASE_HOST = get_key('.env', 'SUPABASE_HOST')
SUPABASE_PASSWORD = get_key('.env', 'SUPABASE_PASSWORD')
SUPABASE_PORT = get_key('.env', 'SUPABASE_PORT')
SUPABASE_DB = get_key('.env', 'SUPABASE_DB')

# interactive shell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Pull fixtures and team data from the API 

In [26]:
# FPL API for fixtures
url = 'https://fantasy.premierleague.com/api/fixtures/'
response = requests.get(url)
fixtures_json = response.json()

# store in pandas DF
fixtures_df = pd.DataFrame(fixtures_json)
fixtures_df.head()

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
0,2367698,NaN,False,False,162,None,0,False,None,12,NaN,3,NaN,[],2,2,93482
1,2367713,NaN,False,False,176,None,0,False,None,4,NaN,13,NaN,[],3,5,93496
2,2367538,1.0,True,True,1,2023-08-11T19:00:00Z,90,False,True,13,3.0,6,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",5,2,93321
3,2367540,1.0,True,True,2,2023-08-12T12:00:00Z,90,False,True,16,1.0,1,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,4,93322
4,2367539,1.0,True,True,3,2023-08-12T14:00:00Z,90,False,True,19,1.0,3,1.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,93323


In [27]:
# FPL API for teams
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
response = requests.get(url)
json = response.json()

# storing json outputs as dataframes
teams_df = pd.DataFrame(json['teams'])
teams_df.head()

,code,draw,form,id,loss,name,played,points,position,short_name,...,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,...,None,False,0,1230,1285,1250,1250,1210,1320,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,...,None,False,0,1115,1175,1130,1190,1100,1160,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,...,None,False,0,1060,1095,1050,1100,1060,1090,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,...,None,False,0,1125,1205,1120,1220,1130,1190,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,...,None,False,0,1165,1210,1120,1200,1210,1240,131


## Join the tables to get team names into the fixtures table

In [28]:
# getting team names into fixtures table
fixtures_df['home_team'] = pd.merge(fixtures_df, teams_df, left_on='team_h', right_on='id', how='left')['name']
fixtures_df['away_team'] = pd.merge(fixtures_df, teams_df, left_on='team_a', right_on='id', how='left')['name']

fixtures_df[fixtures_df['finished'] == True].head()

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id,home_team,away_team
2,2367538,1.0,True,True,1,2023-08-11T19:00:00Z,90,False,True,13,3.0,6,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",5,2,93321,Burnley,Man City
3,2367540,1.0,True,True,2,2023-08-12T12:00:00Z,90,False,True,16,1.0,1,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,4,93322,Arsenal,Nott'm Forest
4,2367539,1.0,True,True,3,2023-08-12T14:00:00Z,90,False,True,19,1.0,3,1.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,93323,Bournemouth,West Ham
5,2367541,1.0,True,True,4,2023-08-12T14:00:00Z,90,False,True,12,1.0,5,4.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,3,93324,Brighton,Luton
6,2367542,1.0,True,True,5,2023-08-12T14:00:00Z,90,False,True,10,1.0,9,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,93325,Everton,Fulham


## Retain only necessary columns

In [29]:
# removing unnecessary columns
fixtures_df = fixtures_df[['id', 'event', 'finished', 'kickoff_time', 'team_a', 'team_a_score', 'team_h', 'team_h_score',
                           'team_h_difficulty', 'team_a_difficulty', 'home_team', 'away_team']]

# convert kickoff time to timestamp
fixtures_df['kickoff_time'] = pd.to_datetime(fixtures_df['kickoff_time'])

fixtures_df.head()

,id,event,finished,kickoff_time,team_a,team_a_score,team_h,team_h_score,team_h_difficulty,team_a_difficulty,home_team,away_team
0,162,NaN,False,NaT,12,NaN,3,NaN,2,2,Bournemouth,Luton
1,176,NaN,False,NaT,4,NaN,13,NaN,3,5,Man City,Brentford
2,1,1.0,True,2023-08-11 19:00:00+00:00,13,3.0,6,0.0,5,2,Burnley,Man City
3,2,1.0,True,2023-08-12 12:00:00+00:00,16,1.0,1,2.0,2,4,Arsenal,Nott'm Forest
4,3,1.0,True,2023-08-12 14:00:00+00:00,19,1.0,3,1.0,2,2,Bournemouth,West Ham


In [30]:
fixtures_df = fixtures_df.rename(columns={'id': 'match_id', 'event': 'gameweek', 'team_a': 'away_team_id', 'team_h': 'home_team_id'})
fixtures_df.head()

,match_id,gameweek,finished,kickoff_time,away_team_id,team_a_score,home_team_id,team_h_score,team_h_difficulty,team_a_difficulty,home_team,away_team
0,162,NaN,False,NaT,12,NaN,3,NaN,2,2,Bournemouth,Luton
1,176,NaN,False,NaT,4,NaN,13,NaN,3,5,Man City,Brentford
2,1,1.0,True,2023-08-11 19:00:00+00:00,13,3.0,6,0.0,5,2,Burnley,Man City
3,2,1.0,True,2023-08-12 12:00:00+00:00,16,1.0,1,2.0,2,4,Arsenal,Nott'm Forest
4,3,1.0,True,2023-08-12 14:00:00+00:00,19,1.0,3,1.0,2,2,Bournemouth,West Ham


## Load into Supabase

In [8]:
# establish connection
conn = psycopg2.connect(
    database=SUPABASE_DB, 
    user=SUPABASE_USER, 
    password=SUPABASE_PASSWORD, 
    host=SUPABASE_HOST, 
    port=SUPABASE_PORT
)

# Setting auto commit true
conn.autocommit = True

# Creating a cursor object using the cursor() method
cursor = conn.cursor()


In [9]:
# Drop the table before creating it again
drop_query = 'DROP TABLE IF EXISTS public.dim_fpl_fixtures;'
cursor.execute(drop_query)

# close the cursor and connection
cursor.close()
conn.close()

In [10]:
# Create SQL alchemy engine
engine_url = 'postgresql://' + SUPABASE_USER + ':' + SUPABASE_PASSWORD + '@' + SUPABASE_HOST + '/' + SUPABASE_DB
engine = create_engine(engine_url)

# Load the table in supabase
fixtures_df.to_sql(
    'dim_fpl_fixtures',
    engine,
    schema='public',
    index=False
)

380

## Combining with player level data

### Add player's next 5 fixtures

In [13]:
# FPL API URL
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
response = requests.get(url)
json = response.json()

# storing json outputs as dataframes
elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])


# Pulling in player position into elements_df
elements_df['position'] = elements_df.element_type.map(elements_types_df.set_index('id').singular_name)
elements_df['team_name'] = elements_df.team.map(teams_df.set_index('id').name)



# Filtering out only the necessary columns
slim_elements_df = elements_df[['id', 'first_name','second_name','web_name','team_name','position','news','selected_by_percent','in_dreamteam',
                                'now_cost','form','points_per_game','minutes','goals_scored','assists','clean_sheets',
                                'goals_conceded','yellow_cards','red_cards','saves','bonus',
                                'transfers_in','starts','value_season','total_points','influence','creativity','threat','ict_index']]

slim_elements_df.rename(columns = {'web_name':'name'}, inplace = True)


# numeric columns:
numeric_cols = ['selected_by_percent','form','points_per_game','value_season','influence','creativity','threat','ict_index']


# convering columns into numeric data type
for col in numeric_cols:
    slim_elements_df[col] = pd.to_numeric(slim_elements_df[col])


# actual cost of the player is now_cost/10
slim_elements_df['actual_cost'] = slim_elements_df['now_cost']/10


# creating additional metrics
slim_elements_df['games_completed'] = slim_elements_df['minutes']/90
slim_elements_df['points_per_90_mins'] = slim_elements_df['total_points']/slim_elements_df['games_completed']
slim_elements_df['ga_per_90_mins'] = (slim_elements_df['goals_scored']+slim_elements_df['assists'])/slim_elements_df['games_completed']
slim_elements_df['goal_contributions'] = (slim_elements_df['goals_scored']+slim_elements_df['assists'])
slim_elements_df['points_per_million'] = slim_elements_df['total_points']/slim_elements_df['actual_cost']

slim_elements_df.head()

,id,first_name,second_name,name,team_name,position,news,selected_by_percent,in_dreamteam,now_cost,...,influence,creativity,threat,ict_index,actual_cost,games_completed,points_per_90_mins,ga_per_90_mins,goal_contributions,points_per_million
0,1,Folarin,Balogun,Balogun,Arsenal,Forward,Transferred to Monaco,0.2,False,44,...,0.0,0.0,0.0,0.0,4.4,0.000000,NaN,NaN,0,0.000000
1,2,Cédric,Alves Soares,Cédric,Arsenal,Defender,,0.4,False,39,...,0.0,0.0,0.0,0.0,3.9,0.000000,NaN,NaN,0,0.000000
2,3,Mohamed,Elneny,M.Elneny,Arsenal,Midfielder,International Duty - Unknown return date,0.1,False,44,...,1.8,0.5,2.0,0.4,4.4,0.011111,360.000000,90.000000,1,0.909091
3,4,Fábio,Ferreira Vieira,Fábio Vieira,Arsenal,Midfielder,Groin Injury - Expected back 04 Feb,0.1,False,54,...,79.4,63.6,77.0,21.9,5.4,2.622222,8.008475,1.525424,4,3.888889
4,5,Gabriel,dos Santos Magalhães,Gabriel,Arsenal,Defender,,17.5,False,50,...,322.2,74.9,112.0,51.0,5.0,16.266667,3.565574,0.061475,1,11.600000


In [69]:
query = """
        with home_fixtures as 
        (
            select 
                a.id
                ,a.name
                ,'Home' as fixture_home_away
                ,b.away_team as opponent
                ,b.kickoff_time
                ,b.gameweek
                ,b.team_h_difficulty as fixture_difficulty_rating
            from 
                slim_elements_df as a
                left join fixtures_df as b on a.team_name = b.home_team
            where
                finished = False
        )
        
        , away_fixtures as 
        (
            select 
                a.id
                ,a.name
                ,'Away' as fixture_home_away
                ,b.home_team as opponent
                ,b.kickoff_time
                ,b.gameweek
                ,b.team_a_difficulty as fixture_difficulty_rating
                
            from 
                slim_elements_df as a
                left join fixtures_df as b on a.team_name = b.away_team
            where
                finished = False
        )

        , final as 
        (
            select
                *
                ,row_number() over (partition by id order by kickoff_time) as fixture_rank
            from
                (
                    select * from home_fixtures
                    union all
                    select * from away_fixtures
                )
            order by
                kickoff_time
        )
        
        select
            *
        from
            final
        where
            fixture_rank <= 5 
            and kickoff_time is not null
        ;
        """

next_fixtures_df = sqldf(query, locals())
next_fixtures_df.head()

,id,name,fixture_home_away,opponent,kickoff_time,gameweek,fixture_difficulty_rating,fixture_rank
0,126,Caicedo,Home,Fulham,2024-01-13 12:30:00.000000,21.0,2,1
1,145,Sanchez,Home,Fulham,2024-01-13 12:30:00.000000,21.0,2,1
2,185,Ampadu,Home,Fulham,2024-01-13 12:30:00.000000,21.0,2,1
3,187,Arrizabalaga,Home,Fulham,2024-01-13 12:30:00.000000,21.0,2,1
4,188,Aubameyang,Home,Fulham,2024-01-13 12:30:00.000000,21.0,2,1


In [86]:
next_fixtures_df.sort_values(by=['id', 'fixture_rank']).head(10)

,id,name,fixture_home_away,opponent,kickoff_time,gameweek,fixture_difficulty_rating,fixture_rank,opponent_order
394,1,Balogun,Home,Crystal Palace,2024-01-20 12:30:00.000000,21.0,2,1,Opponent 1
773,1,Balogun,Away,Nott'm Forest,2024-01-30 19:30:00.000000,22.0,2,2,Opponent 2
2177,1,Balogun,Home,Liverpool,2024-02-04 16:30:00.000000,23.0,4,3,Opponent 3
2863,1,Balogun,Away,West Ham,2024-02-11 14:00:00.000000,24.0,3,4,Opponent 4
3128,1,Balogun,Away,Burnley,2024-02-17 15:00:00.000000,25.0,2,5,Opponent 5
395,2,Cédric,Home,Crystal Palace,2024-01-20 12:30:00.000000,21.0,2,1,Opponent 1
774,2,Cédric,Away,Nott'm Forest,2024-01-30 19:30:00.000000,22.0,2,2,Opponent 2
2178,2,Cédric,Home,Liverpool,2024-02-04 16:30:00.000000,23.0,4,3,Opponent 3
2864,2,Cédric,Away,West Ham,2024-02-11 14:00:00.000000,24.0,3,4,Opponent 4
3129,2,Cédric,Away,Burnley,2024-02-17 15:00:00.000000,25.0,2,5,Opponent 5
